# 1. Cài đặt thư viện và tải data

In [1]:
# load pre-trained BERT và tune
!pip install transformers

In [2]:
!git clone https://github.com/duyvuleo/VNTC.git

Cloning into 'VNTC'...
remote: Enumerating objects: 39, done.
remote: Total 39 (delta 0), reused 0 (delta 0), pack-reused 39
Unpacking objects: 100% (39/39), 160.90 MiB | 10.94 MiB/s, done.


In [3]:
%cd /kaggle/working/VNTC/Data/10Topics/Ver1.1
!apt install unrar

/kaggle/working/VNTC/Data/10Topics/Ver1.1



The following NEW packages will be installed:
  unrar
0 upgraded, 1 newly installed, 0 to remove and 62 not upgraded.
Need to get 113 kB of archives.
After this operation, 406 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 unrar amd64 1:5.6.6-2build1 [113 kB]
Fetched 113 kB in 8s (13.5 kB/s)

78Selecting previously unselected package unrar.
(Reading database ... 107701 files and directories currently installed.)
Preparing to unpack .../unrar_1%3a5.6.6-2build1_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########...............................................] 8Unpacking unrar (1:5.6.6-2build1) ...
7Progress: [ 40%] [#######################...................................] 8Setting up unrar (1:5.6.6-2build1) ...
7Progress: [ 60%] [##################################........................] 8update-altern

In [4]:
!unrar x Test_Full.rar /kaggle/working > /dev/null
!unrar x Train_Full.rar /kaggle/working > /dev/null
%cd /kaggle/working
!ls

/kaggle/working
Test_Full  Train_Full  VNTC  __notebook__.ipynb


# 2. Data processing

In [5]:
import os
import torch
from typing import List, Tuple, Dict
from tqdm import tqdm

In [6]:
TRAIN_PATH = "/kaggle/working/Train_Full"
TEST_PATH = "/kaggle/working/Test_Full"

def read_txt(path):
    with open(path, 'r', encoding='utf-16') as f:
        data = f.read()
    return data

def make_data(root_path):
    texts = []
    labels = []
    
    for root, dirs, files in tqdm(os.walk(root_path)):
        for file_name in files:
            try:
                file_path = os.path.join(root, file_name)
                content = read_txt(file_path)
                label = root.split(os.path.sep)[-1]
                texts.append(content)
                labels.append(label)
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
    
    return texts, labels

text_train, label_train = make_data(TRAIN_PATH)
text_test, label_test = make_data(TEST_PATH)

11it [00:01,  6.90it/s]
11it [00:02,  4.59it/s]


In [7]:
print(text_train[0], label_train[0])
print(len(text_train), len(text_test))

Phúc hãy quên đi cô nàng dễ thay đổi đó
Theo tôi cô ấy không có lập trường vững vàng, tình cảm dễ thay đổi, và hơn nữa cô ấy không tôn trọng anh cũng như người yêu cũ của cô ta. Nếu là người yêu của anh, cô ấy sẵn sàng từ bỏ anh bất cứ lúc nào nếu như phải lòng một người đàn ông nào khác.
From: Van Loi To: webmaster@vnexpress.net Sent: Friday, September 16, 2005 2:28 PM Subject: Gop y cung anh Phuc 
Gởi anh Phúc! 
Tôi đọc được bài viết của anh, nhận thấy rằng anh đã đặt tình yêu của mình cho một người không biết xem trọng tình yêu. Khi một người con gái đã có một cuộc tình dài 4 năm và đã đi đến quyết định làm đám cưới, được sự đồng ý của cả hai gia đình, trong cuộc tình này không ít thì nhiều cũng có được tình yêu từ hai phía và mới quyết định làm đám cưới, nhưng cô ấy đành từ bỏ những chuyện ấy để đến với anh. Trong khi cô ấy chỉ quen anh chưa đầy một năm, tức là lúc này cô ấy vừa quen cả hai người.
Vì nhận thấy anh có cái gì đó hơn người yêu của cô ấy, cô ấy sẵn sàng từ bỏ theo anh.

**Encode các labels**

In [8]:
from sklearn.preprocessing import LabelEncoder

lb_encoder = LabelEncoder()
label_train_encoded = lb_encoder.fit_transform(label_train)
label_test_encoded = lb_encoder.transform(label_test)

print(lb_encoder.classes_)

['Chinh tri Xa hoi' 'Doi song' 'Khoa hoc' 'Kinh doanh' 'Phap luat'
 'Suc khoe' 'The gioi' 'The thao' 'Van hoa' 'Vi tinh']


In [9]:
# Chia tập train thành tập train và valid
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(text_train, label_train_encoded, test_size=0.1, stratify=label_train_encoded)

**Tạo dataloader**

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base')

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
print("Model max_length: ", tokenizer.model_max_length)

Model max_length:  256


In [12]:
import torch
from torch.utils.data import TensorDataset, DataLoader

def make_dataloader(texts, labels, BATCH_SIZE=32):
    # Mã hóa văn bản thành input_ids sử dụng tokenizer
    input_ids = tokenizer.batch_encode_plus(texts, padding=True, truncation=True, return_tensors='pt')['input_ids']
    labels = torch.tensor(labels)
    
    dataset = TensorDataset(input_ids, labels)
    return DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True)

train_dataloader = make_dataloader(X_train, y_train)
valid_dataloader = make_dataloader(X_val, y_val)
test_dataloader = make_dataloader(text_test, label_test_encoded)

In [13]:
print(len(train_dataloader))

950


# Model

In [14]:
class PhoBERT_classification(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.phobert = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels = num_classes)
        
        for param in self.phobert.roberta.parameters():
            param.requires_grad = False
            
    def forward(self, input_ids, labels):
        out = self.phobert(input_ids, labels = labels)
        probs = torch.softmax(out['logits'], dim=-1)
        preds = torch.argmax(probs, dim=-1).cpu().detach().numpy()
        out['preds'] = preds
        return out

In [15]:
num_classes = len(lb_encoder.classes_)

model = PhoBERT_classification(num_classes)

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['

In [16]:
def get_num_parameters(module: torch.nn.Module) -> Dict[str, int]:
    """
    Get the number of trainable, fixed and total parameters of a pytorch module.
    :param module: pytorch module
    :return: dict containing number of parameters
    """
    trainable = 0
    fixed = 0
    for p in module.parameters():
        if p.requires_grad:
            trainable += p.numel()
        else:
            fixed += p.numel()
    return {"trainable": trainable, "fixed": fixed, "total": trainable + fixed}

get_num_parameters(model)

{'trainable': 598282, 'fixed': 134407680, 'total': 135005962}

# Training model

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [18]:
LR = 1e-4
EPOCHS = 5
LOG_STEP = 100

In [19]:
model.to(device)

PhoBERT_classification(
  (phobert): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(64001, 768, padding_idx=1)
        (position_embeddings): Embedding(258, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0): RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense

In [20]:
from sklearn.metrics import classification_report

def train_model(optimizer):
    total_loss = 0
    for i in range(EPOCHS):
        print(f'------------ Epoch {i+1} -----------------')
        print('Training ...')
        model.train()
        for step, (input_ids, labels) in tqdm(enumerate(train_dataloader)):
            optimizer.zero_grad()
            input_ids = input_ids.to(device)
            labels = labels.to(device)

            out = model(input_ids, labels)
            loss = out['loss']
            loss.backward()
            optimizer.step()

            total_loss += loss.detach().cpu().item()

        preds = []
        truths = []
        print('Running Validation...')
        model.eval()
        valid_loss = 0
        with torch.no_grad():
            for step, (input_ids, labels) in tqdm(enumerate(valid_dataloader)):
                input_ids = input_ids.to(device)
                labels = labels.to(device)

                out = model(input_ids, labels)
                loss = out['loss']
                valid_loss += loss.detach().cpu().item()
                preds.extend(out['preds'])
                truths.extend(labels.detach().cpu().numpy())
        print(classification_report(truths, preds, target_names=lb_encoder.classes_))
        print(f"Valid loss: {valid_loss / len(valid_dataloader)}")

In [21]:
# optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# train_model(optimizer)

In [22]:
from transformers import AdamW

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
    'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
    'weight_decay': 0},
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5, correct_bias=False)

train_model(optimizer)

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


------------ Epoch 1 -----------------
Training ...


950it [03:54,  4.05it/s]


Running Validation...


106it [00:24,  4.26it/s]


...

                  precision    recall  f1-score   support

Chinh tri Xa hoi       0.56      0.76      0.64       522
        Doi song       0.74      0.73      0.74       316
        Khoa hoc       0.84      0.57      0.68       182
      Kinh doanh       0.79      0.63      0.70       255
       Phap luat       0.76      0.79      0.77       387
        Suc khoe       0.74      0.78      0.76       338
        The gioi       0.81      0.81      0.81       290
        The thao       0.97      0.93      0.95       530
         Van hoa       0.83      0.74      0.78       308
         Vi tinh       0.94      0.74      0.83       248

        accuracy                           0.77      3376
       macro avg       0.80      0.75      0.77      3376
    weighted avg       0.79      0.77      0.77      3376

Valid loss: 0.7563440288012883
------------ Epoch 5 -----------------
Training ...


950it [03:54,  4.06it/s]


Running Validation...


106it [00:24,  4.25it/s]

                  precision    recall  f1-score   support

Chinh tri Xa hoi       0.57      0.75      0.65       522
        Doi song       0.77      0.74      0.76       316
        Khoa hoc       0.82      0.61      0.70       182
      Kinh doanh       0.80      0.64      0.71       255
       Phap luat       0.74      0.80      0.77       387
        Suc khoe       0.74      0.80      0.77       338
        The gioi       0.84      0.80      0.82       290
        The thao       0.98      0.92      0.95       530
         Van hoa       0.80      0.76      0.78       308
         Vi tinh       0.92      0.75      0.83       248

        accuracy                           0.78      3376
       macro avg       0.80      0.76      0.77      3376
    weighted avg       0.79      0.78      0.78      3376

Valid loss: 0.7041297301931201


In [23]:
torch.save(model.state_dict(), f'phobert_text_classification.pth') 

# Testing model

In [24]:
def test_model(data_loader):
    preds = []
    truths = [] 
    model.eval()
    with torch.no_grad():
        for step, (input_ids, labels) in tqdm(enumerate(valid_dataloader)):
            input_ids = input_ids.to(device)
            labels = labels.to(device)

            out = model(input_ids, labels)
            loss = out['loss']
            
            preds.extend(out['preds'])
            truths.extend(labels.detach().cpu().numpy())
            
    print(classification_report(truths, preds, target_names=lb_encoder.classes_))
    
test_model(test_dataloader)

106it [00:24,  4.25it/s]

                  precision    recall  f1-score   support

Chinh tri Xa hoi       0.57      0.75      0.65       522
        Doi song       0.77      0.74      0.76       316
        Khoa hoc       0.82      0.61      0.70       182
      Kinh doanh       0.80      0.64      0.71       255
       Phap luat       0.74      0.80      0.77       387
        Suc khoe       0.74      0.80      0.77       338
        The gioi       0.84      0.80      0.82       290
        The thao       0.98      0.92      0.95       530
         Van hoa       0.80      0.76      0.78       308
         Vi tinh       0.92      0.75      0.83       248

        accuracy                           0.78      3376
       macro avg       0.80      0.76      0.77      3376
    weighted avg       0.79      0.78      0.78      3376

